In [1]:
# importing module
import sys
import os
from pathlib import Path
sys.path.append(str(Path.cwd().parent))

In [2]:
import numpy as np
import pandas as pd
import sklearn
import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use('seaborn')
params = {
    'figure.figsize': (20, 15),
    'legend.fontsize': 'x-large',
    'axes.labelsize': 'x-large',
    'axes.titlesize':'x-large',
    'xtick.labelsize':'x-large',
    'ytick.labelsize':'x-large',
    'axes.titlesize': 'x-large',
}
plt.rcParams.update(params)

In [8]:
import pathlib

def load_ml_dataset():
    DATASET_PATH = pathlib.Path(pathlib.Path.cwd()).parent / "data"
    DATASET_NAME = "ML-CUP19-TR.csv"
    print("loading from: ")
    print(DATASET_PATH / DATASET_NAME)
    return pd.read_csv(DATASET_PATH / DATASET_NAME, sep=',', comment='#', skiprows=7, header=None, index_col=0)

def load_airfoil_dataset():
    DATASET_PATH = "data"
    DATASET_NAME = "airfoil_self_noise.csv"
    df = pd.read_csv(DATASET_PATH + "/" + DATASET_NAME)
    df.columns = ['Frequency (HZ)', 'Angle of attack (deg)', 'Chord length (m)', 'Free-stream velocity (m/s)', 'Suction side displacement thickness (m)', 'Scaled sound pressure level (db)']
    return df

def load_california_dataset():
    from sklearn.datasets.california_housing import fetch_california_housing
    data = fetch_california_housing()
    return pd.DataFrame(data.data, columns=data.feature_names)
    
    
    

In [9]:
df = load_airfoil_dataset()

In [10]:
df

,Frequency (HZ),Angle of attack (deg),Chord length (m),Free-stream velocity (m/s),Suction side displacement thickness (m),Scaled sound pressure level (db)
0,800,0.0,0.3048,71.3,0.002663,126.201
1,1000,0.0,0.3048,71.3,0.002663,125.201
2,1250,0.0,0.3048,71.3,0.002663,125.951
3,1600,0.0,0.3048,71.3,0.002663,127.591
4,2000,0.0,0.3048,71.3,0.002663,127.461
...,...,...,...,...,...,...
1498,2500,15.6,0.1016,39.6,0.052849,110.264
1499,3150,15.6,0.1016,39.6,0.052849,109.254
1500,4000,15.6,0.1016,39.6,0.052849,106.604
1501,5000,15.6,0.1016,39.6,0.052849,106.224


In [8]:
def separate_feature(dataset, nlast=1):
    """separates the features from the target variables.
    Params:
        dataset  -- the dataset.
        nlast    -- number of columns containing the target variables.
    Returns:
        features, targets
    """
    target_points = df[df.columns[-nlast:]]
    target_points.columns = ['X', 'Y']
    feature_points = df[df.columns[:-nlast]]
    feature_points.columns = [str(i+1) for i in range(len(feature_points.columns))]
    return feature_points, target_points

In [39]:
def randomsample(mat, n):
    """samples a random subset of size n from a matrix."""
    r,c = mat.shape
    assert(r >= n)
    M = mat.to_numpy() if type(mat) is pd.core.frame.DataFrame else mat
    return M[np.random.choice(r, n, replace=False),:]

# TODO
* sampling dati dal dataset originale
* 

In [42]:
samples = [randomsample(df, i) for i in [10,100,1000]]